# RadxRate Tutorial

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local particle identification (PID) category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

A visual comparison of RadxKdp, RadxPid, and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxRate parameter file links to the specific differential phase (KDP), PID, and rate parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the echo tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "lrose-hub" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/RadxRate_structure.png" width="500"/>
</div>

#### Download raw data and parameter files

Raw data files that are provided:
* gfsanl_4_2018091418.g2.tar
* KMHX20180914_191822_V06.ar2v

Parameter files (included in this tutorial):

*Note: The parameter files have already been modified to run straight out of the box.* 
* Grib2toMdv params
* Mdv2SoundingSpdb params
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (S-band, simultaneous transmit)

### 2. Prepare data for analysis

* Convert Nexrad level 2 to CfRadial
    * RadxConvert
* Retrieve sounding from GFS analysis, which is needed to estimate the melting level in the PID analysis
    * Grib2toMdv, Mdv2SoundingSpdb

### 3. Run RadxRate

* Calculate KDP, run PID algorithm, estimate three-dimensional precipitation rates
    * RadxRate

### 4. Plot PID and Rate

* Visualize results of RadxRate analysis using [Py-ART](https://arm-doe.github.io/pyart/)

---

For this tutorial, the parameter files have already been created and populated with the appropriate parameters. However, the default parameter files can be saved using the following commands.

In [ ]:
# # create original parameter files
# !Grib2toMdv -print_params > /home/jovyan/ams2023/params/user/Grib2toMdv_params_user
# !Mdv2SoundingSpdb -print_params > /home/jovyan/ams2023/params/user/Mdv2SoundingSpdb_params_user
# !RadxRate -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user
# !RadxRate -print_params_kdp > /home/jovyan/ams2023/params/user/RadxRate_Kdp_params_user
# !RadxRate -print_params_pid > /home/jovyan/ams2023/params/user/RadxRate_Pid_params_user
# !RadxRate -print_params_rate > /home/jovyan/ams2023/params/user/RadxRate_rate_params_user

# to create new parameter files that use parameters from another file
# !RadxRate -params /path/to/params -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user

# for command line options
# !RadxRate -h


# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook on the LROSE Gateway JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR and RAW_DIR to work on your personal machine. Note that raw data is currently provided through the Gateway. We're working to set up more permanent storage for offline users - in the meantime, contact the Gateway team to request the data.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RAW_DIR: the directory containing the raw data used in tutorials
* RADAR_NAME: the name of the radar used in this tutorial

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
os.environ['RADAR_NAME'] = 'KMHX'
base_dir = os.environ['BASE_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME

## Set up directories

We need to set up the required data directories. The raw radar data and GFS analysis is located within the share directory on the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make a directory for all the data files in the echo tutorial (raw and analysis)
!rm -rf ${BASE_DIR}/data/echo
!mkdir -p ${BASE_DIR}/data/echo

## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/echo/raw
!mkdir ${BASE_DIR}/data/echo/raw

## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/echo/sounding
!mkdir ${BASE_DIR}/data/echo/sounding


We can examine the raw radar file using RadxPrint. RadxPrint can read CfRadial and other raw radar formats supported by LROSE that can later be converted by RadxConvert. It will also provide information on radar variables and sweep information. For example, we can look at the variables using the following command, piping the output into the head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ${RAW_DIR}/echo/KMHX20180914_191822_V06.ar2v | head -50


# 2. Prepare data for analysis
## Convert Nexrad Level II to CfRadial files

Now that we have the raw radar data, we convert it into CfRadial using RadxConvert so that the other applications can read the data. Note that in the vast majority of cases, RadxConvert can be run without a parameter file, including in this case. However, there are instances where a parameter file is useful.

Here, the *-f* flag provides the path to the raw radar file and the *-outdir* flag tells RadxConvert where the CfRadial file will be written. 

In [ ]:
!rm -rf ${BASE_DIR}/data/echo/CfRadial
!mkdir ${BASE_DIR}/data/echo/CfRadial
!RadxConvert -f ${RAW_DIR}/echo/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/echo/CfRadial


## Untar GFS analysis grib2 files for sounding
We use the GFS analysis to obtain a sounding for the PID algorithm. We need to untar the files, convert to a gridded file format (Grib2toMdv), then extract the sounding and store in in the Spdb format (Mdv2SoundingSpdb).

In [ ]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/echo/sounding/grib
!mkdir ${BASE_DIR}/data/echo/sounding/grib

# extract file from tape archive (tar) and place in the grib directory
!tar -xvf ${RAW_DIR}/echo/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/echo/sounding/grib/


## Convert from Grib2 (model analysis) to MDV

The program to retrieve the model sounding requires a gridded file format called MDV. Grib2toMdv converts the GFS analysis from GRIB to MDV. Grib2toMdv needs to know where to write the gridded data files, the name of the fields (the defaults are often sufficient), how the grid should be remapped (e.g., map projection), the size of the new grid, and the vertical levels.

**Important Grib2toMdv Parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 61 |Optional if directory is specified in command line.|
|write_non_forecast| 330 |Specify if output should be written in a non-forecast directory structure (*TRUE* or *FALSE*, usually *TRUE*).|
|non_forecast_mdv_url| 338 |Absolute path where MDV files are written.|
|output_fields| 518 |List of output fields, where param matches GRIB files and mdv_name matches Mdv2SoundingSpdb parameter file (defaults are usually sufficient).|
|remap_output| 672 |Specify whether to remap the output (*TRUE* or *FALSE*, usually *TRUE*).|
|out_projection_info| 708 |Set map projection type and radar location if necessary.|
|out_grid_info| 740 |User-specified domain surrounding the region of interest and radar(s). Parameters: nx/ny - number of points, minx/miny - bottom left lat/lon (deg), dx/dy - delta lon/lat (deg).|
|height_levels| 818 |User-specified height levels, optional.|

    
Here, the *-params* flag directs Grib2toMdv to the parameter file and *-f* points to the raw GRIB file path. This can also be specified in the parameter *input_dir*.


In [ ]:
# Grib2 -> MDV
!Grib2toMdv -params ${BASE_DIR}/params/echo/Grib2toMdv_params -f ${BASE_DIR}/data/echo/sounding/grib/*.grb2


## Convert from MDV (gridded data) to SoundingSpdb

Now that we have model analysis in a small grid around the radar(s), we can extract the sounding from the radar location and store it in a database format (Spdb). Mdv2SoundingSpdb needs the variable names in the MDV files, the geographic location of the radars, and the path where the Spdb files will be writen.

**Mdv2SoundingSpdb**
| Parameter | Line # | Description |
| --- | --- | --- |
|temp_field_name| 125 |Temperature variable name.|
|rh_field_name| 145 |RH variable name.|
|u_wind_field_name| 173 |U wind variable name.|
|v_wind_field_name| 183 |V wind variable name.|
|pressure_field_name| 205 |Pressure variable name.|
|height_field_name| 217 |Height variable name.|
|station_locations| 247 |Geographic location of the radar(s): radar lat, lon, altitude.|
|ouput_url| 268 |Absolute path where Spdb files are written.|


We have populated the radar information in the parameter files. The commands below are some ways to get the station location information for the files in the tutorial. 

In [ ]:
# !RadxPrint -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc | head -30
# !RadxPrint -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc | grep -i latitudeDeg | sed 's/^.*: //'
# !RadxPrint -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc | grep -i longitudeDeg | sed 's/^.*: //'
# !RadxPrint -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc | grep -i altitudeKm | sed 's/^.*: //'
# !RadxPrint -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc | grep -i sensorHtAglM | sed 's/^.*: //'


Here, the *-params* flag directs Mdv2SoundingSpdb to the parameter file and *-f* points to the raw GRIB file.

In [ ]:
# convert Mdv to Spdb
!Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180914/*.mdv*
!Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180915/*.mdv*


We can check the sounding quality using SpdbQuery, which prints out a summary of the sounding data. Here, *-url* indicates the directory where the Spdb files are located and *-start* and *-end* provide the date range over which we want to inspect the sounding information. 

In [ ]:
!SpdbQuery -url ${BASE_DIR}/data/echo/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"


# 3. Setup steps complete, run RadxRate

With the radar data and point sounding, we can now run RadxRate. First, KDP is calculated. Then polarimetric radar data, calculated KDP, and sounding data are used to run PID, which determines the dominant hydrometeor at each range gate. Finally, the 3D field of hourly rain rates are calculated from polarimetric radar and PID data.

Running RadxRate requires one main parameter file on the command line. That parameter file contains  links to the KDP, PID, and rain rate coefficient parameter files. The RadxRate PID parameter file contains the link to the fuzzy logic PID thresholds file. Because there are so many parameters, we have provided them at the end of this notebook. Please refer to [this link](#RadxRad-parameters).

RadxRate requires the field names in the input CfRadial file, whether SNR and LDR are available, the method for KDP calculation, how to calculate the PID, how to calculate the precipitation rates (e.g., equations and coefficients), whether to use attenuation-correction fields, the variables to be written, and whether to censor non-weather gates. 

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
!RadxRate -params ${BASE_DIR}/params/echo/RadxRate_main_params -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/echo/rate


# 4. Plot PID and rate for NEXRAD Morehead City radar (KMHX)

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/echo/rate/20180914/"
file = "cfrad.20180914_191822.077_to_20180914_192511.621_KMHX_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


# 5. Extras

<a id=’RadxRate-parameters’></a>
## **RadxRate parameters**

Below is a list of important RadxRate parameters. The line numbers are accurate as of Jan 2025 for the Colette release of LROSE. 

**RadxRate main parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 98 |Path to input data, if not already specified on the command line.|
|SNR_available| 160 |Specify if SNR is available.|
|SNR_field_name| 170 |SNR variable name.|
|DBZ_field_name| 192 |DBZ variable name.|
|ZDR_field_name| 200 |ZDR variable name.|
|PHIDP_field_name| 208 |PHIDP variable name.|
|RHOHV_field_name| 216 |RHOHV variable name.|
|LDR_available| 224 |Specify if LDR is available.|
|LDR_field_name| 232 |LDR variable name.|
|kdp_params_file_path| 270 |Path to Kdp-specific parameter file.|
|pid_params_file_path| 287 |Path to PID-specific parameter file.|
|PID_use_attenuation_corrected_fields| 311 |Specify whether to use attenuation-corrected DBZ and ZDR in PID.|
|RATE_params_file_path| 328 |Path to precipitation rate coefficients parameter file.|
|RATE_use_attenuation_corrected_fields| 352 |Specify whether to use attenuation-corrected DBZ and ZDR in precipitation rate calculation.|
|output_dir| 757 |Path where output files are written, if not already specified on command line with -outdir.|
    
**RadxRate Kdp_specific parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|KDP_fir_filter_len| 63 |Filter length used for KDP calculation.|
|KDP_psob_method| 108 |Specify method to remove phase shift on backscatter.|
          
**RadxRate Pid_specific parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|PID_thresholds_file_path| 25 |Path to fuzzy logic PID thresholds file.|
|PID_use_soundings_from_spdb| 259 |Specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file.|
|PID_sounding_spdb_url| 267 |Path to Spdb soundings.|
|PID_sounding_location_name| 289 |Name of sounding location.|
          
**RadxRate Rate_specific parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|RATE_zh_aa| 136 |R(Z) coefficients.|
|RATE_zh_bb| 144 |R(Z) coefficients.|
|RATE_zh_aa_snow| 160 |R(Z) coefficients in snow.|
|RATE_zh_bb_snow| 168 |R(Z) coefficients in snow.|
|RATE_zzdr_aa| 184 |R(Z, ZDR) coefficients.|
|RATE_zzdr_bb| 192 |R(Z, ZDR) coefficients.|
|RATE_zzdr_cc| 200 |R(Z, ZDR) coefficients.|
|RATE_kdp_aa| 216 |R(KDP) coefficients.|
|RATE_kdp_bb| 224 |R(KDP) coefficients.|
|RATE_kdpzdr_aa| 241 |R(KDP, ZDR) coefficients.|
|RATE_kdpzdr_bb| 249 |R(KDP, ZDR) coefficients.|
|RATE_kdpzdr_cc| 257 |R(KDP, ZDR) coefficients.|
      
**Pid Thresholds parameters**

The thresholds files are radar dependent. In this notebook, NEXRAD is S-band simultaneous h/v transmission and recieving

Other parameter files can be found [here](http://wiki.lrose.net/index.php/RadxPid#PID_thresholds_file).
